In [1]:
import numpy as np
from copy import deepcopy
with open('input.txt') as fl:
    maze = np.array([[v for v in ln.strip() ] for ln in fl.readlines()])

In [2]:
def pretty_print(routes):
    all_routes = set()
    for route in routes:
        all_routes.update(route['history'])
    for i in range(maze.shape[0]):
        s = ''
        for j in range(maze.shape[1]):
            if (i,j) in all_routes:
                s += 'O'
            else:
                s += maze[i][j]
        print(s)

In [3]:
dirs = {'^':(-1,0), 'v':(1,0), '>':(0,1), '<':(0,-1)}
uphill = {'^':'v', 'v':'^', '>':'<', '<':'>'}

In [4]:
# last junc ... (pos, dir)
# cache, bad junc (pos, dir, block)

In [5]:
# part1
start, end = (0,1), (maze.shape[0]-1, np.where(maze[-1, :]=='.')[0][0])
route = {'position':start, 'prev_pos':(-99999,-99999), 'history':set([start]), 'tile':'.'}
routes = [route]
bad_routes = []
completed_routes = []
while len(routes) > 0:
    next_routes = []
    for route in routes:
        # if on a slope, we simply follow
        moved = False
        if route['tile'] in dirs:
            i0, j0 = route['position']
            di, dj = dirs[route['tile']]
            i, j = i0+di, j0+dj
            if (i,j) not in route['history']:
                route['position'] = (i,j)
                route['history'].add((i,j))
                route['tile'] = maze[i][j]
                next_routes.append(route)
                moved=True
        # otherwise, check all directions and maybe do more than one
        else:
            i0, j0 = route['position']    
            good_dirs = []
            for d, (di, dj) in dirs.items():
                i, j = i0+di, j0+dj
                if not (maze[i][j] == '#' or maze[i][j] == uphill[d]):
                    if (i,j) != route['prev_pos']:
                        if (i,j) not in route['history']:
                            good_dirs.append((d, (i,j)))

            for d, (i,j) in good_dirs:
                new_route = deepcopy(route) if len(good_dirs)>1 else route
                new_route['prev_pos'] = new_route['position']
                new_route['position'] = (i,j)
                new_route['history'].add((i,j))
                new_route['tile'] = maze[i][j]
                next_routes.append(new_route)
                moved=True
                
        if not moved:
            bad_routes.append(route)
            
    routes = [r for r in next_routes if r['position'] != end]
    completed_routes.extend([r for r in next_routes if r['position'] == end])
max([len(r['history']) for r in completed_routes]) - 1   

2230

In [6]:
# part 2 
start, end = (0,1), (maze.shape[0]-1, np.where(maze[-1, :]=='.')[0][0])
def get_adjacent_juncs(i,j):
    start = (i,j)
    route = {'position':start, 'prev_pos':(-99999,-99999), 'history':set([start]), 'tile':'.'}
    routes = [route]
    completed_routes = []
    it = 0
    
    while len(routes) > 0:
        next_routes = []
        for route in routes:
            i0, j0 = route['position']    
            good_dirs = []
            for d, (di, dj) in dirs.items():
                i, j = i0+di, j0+dj
                if 0<=i<maze.shape[0] and 0<=j<maze.shape[1] and not maze[i][j] == '#':
                    if (i,j) != route['prev_pos']:
                        if (i,j) not in route['history']:
                            good_dirs.append((d, (i,j)))  
            #print(it, good_dirs)

            if (it>0 and len(good_dirs) > 1) or (i0, j0) == end:
                completed_routes.append(route)
            else:
                for d, (i,j) in good_dirs:
                    new_route = deepcopy(route) if len(good_dirs)>1 else route
                    new_route['prev_pos'] = new_route['position']
                    new_route['position'] = (i,j)
                    new_route['history'].add((i,j))
                    new_route['tile'] = maze[i][j]
                    next_routes.append(new_route)
            routes = next_routes
        it += 1    
        
    return [(r['position'], len(r['history'])-1) for r in completed_routes]
        

In [7]:
# build the weighted graph
graph = {}
p_len = -1
all_juncs = set([start])
new_juncs = set([start])
while len(new_juncs) > 0:
    juncs = new_juncs
    new_juncs = set([])
    for junc in juncs:
        res = get_adjacent_juncs(*junc)
        for (i,j), dst in res:
            if (i,j) not in all_juncs:
                new_juncs.add((i,j))
                all_juncs.add((i,j))
        graph[junc] = res


In [8]:
paths = [([start,], 0)]
new_paths = paths
max_dist = 0
while len(new_paths) > 0:
    paths = new_paths
    new_paths = []
    for path, total_dist in paths:
        junc = path[-1]
        prev_juncs = path[:-1]
        for next_junc, dist in graph[junc]:
            if next_junc not in prev_juncs:
                new_paths.append((path + [next_junc], total_dist+dist))
                if next_junc == end:
                    max_dist = max(max_dist, total_dist+dist)
max_dist

6542